In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from joblib import dump, load

In [7]:
X = np.load('features.npy')
y = np.load('group.npy')
features = ['aspect', 'curvature', 'elevation', 'morphology', 'slope', 'tree-cover', 'patches']

In [8]:
estimators = [
    ('rfc', RandomForestClassifier()),
#     ('lda', LinearDiscriminantAnalysis()),
#     ('qda', QuadraticDiscriminantAnalysis()),
#     ('log', LogisticRegression())
]

params = {
    'rfc': {
        'n_estimators': [200],
        'max_depth': [20],
        'min_samples_split': [2000],
        'class_weight': ['balanced_subsample']
    },
    
    'lda': {
        'n_components': [2,]
    },
    
    'qda': {
        
        
    },
    
    'log': {
        'penalty' : ['l1', 'l2'],
        'C': [1.0, 0.1, 0.01, 0.001],
        'class_weight': ['balanced']
    }
    
}

In [9]:
#X_ = StandardScaler().fit_transform(X[:, [True, False,False, True,True,True,True,True,False]])
X_ = X[:, [True, False, False, True, True, True, True, True, False]]
y = y.astype(int)

In [ ]:
results = dict()
for cl_name, clf in estimators:
    gcv = GridSearchCV(clf, param_grid=params[cl_name], n_jobs=14, scoring='balanced_accuracy', verbose=1, cv=5)
    gcv.fit(X_, y)
    print(f"Resutls for {cl_name}.")
    print(f"Balanced accuracy score: {gcv.best_score_}")
    print(f"Confusion matrix: {confusion_matrix(gcv.best_estimator_.predict(X_), y)}")
    results[cl_name] = gcv

In [ ]:
gcv.best_params_

In [ ]:
for f, imp in zip(features[:-1], results['rfc'].best_estimator_.feature_importances_):
    print(f"feature {f}: {imp}.")

In [ ]:
#X_ = StandardScaler().fit_transform(X[:, [True, False,False, True,True,True,True,True,False]])
#y = y.astype(int)

In [10]:
# features = ['aspect', 'curvature-plan', 'curvature-prof', 'curvature', 'elevation', 'morphology', 'slope', 'tree-cover', 'patches']
features = np.array(['aspect',  'curvature', 'elevation', 'morphology', 'slope', 'tree-cover'])

In [11]:
n_features = len(features)

In [12]:
n_features

6

In [13]:
X_.shape

(924000, 6)

In [18]:
Counter(y)

Counter({0: 901495, 1: 22505})

In [23]:
TP = Counter(y)[1] / 2
TN = Counter(y)[0] / 2
FP = Counter(y)[0] / 2
FN = Counter(y)[1] / 2

In [24]:
(TP/(TP+FN) + TN/(TN+FP))/2

0.5

In [25]:
results = []
clf = RandomForestClassifier(**{'class_weight': 'balanced_subsample',
 'max_depth': 20,
 'min_samples_split': 4000,
 'n_estimators': 10})
clf.fit(X_, y)

RandomForestClassifier(class_weight='balanced_subsample', max_depth=20,
                       min_samples_split=4000, n_estimators=10)

In [28]:
dump(clf, 'rf_clf.joblib') 

['rf_clf.joblib']

In [ ]:
results = []
clf = RandomForestClassifier(**{'class_weight': 'balanced_subsample',
 'max_depth': 20,
 'min_samples_split': 4000,
 'n_estimators': 10})
for j in range(2, 2 ** n_features - 1):
    print(j)
    mask = np.array(list(format(j, f'#0{n_features + 2}b')[2:]), dtype=int).astype(bool)
    if sum(mask) > 2:
        errors = cross_val_score(clf, X_[:, mask], y, n_jobs=14,scoring='balanced_accuracy')
        results.append((mask, errors))
        # print(f"Trying features {np.array(features)[mask]}: mean={errors.mean()}, std={errors.std()}.")

In [ ]:
features  

In [ ]:
for ffs, vals in sorted(results, key=lambda x: x[1].mean(), reverse=True):
    print(features[ffs], vals.mean())

In [ ]:
cross_val_score(clf, X_, y, n_jobs=14,scoring='balanced_accuracy')

In [ ]:
comm = []
for ffs, vals in filter(lambda x: x[1].mean()>0.69,results):
    comm.append(features[ffs].tolist())

In [ ]:
from functools import reduce

In [ ]:
reduce(lambda x, y: x.intersection(y), map(set, comm))

{'aspect', 'morphology', 'tree-cover'} -- эти предикторы всегда были в лучших результатах. Они обязательно должны быть, чтобы попытка угадать упадет ли лес в данной точке была хотя бы немного успешной... фактически это общие предикторы для все лучших случаев, т.е. тех случаев, у которых средняя точность, вычисленная по пяти случайным разбиением исходных данных на тестовую и проверочную, оказалась выше 0.69. 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")


# Compute the correlation matrix
corr = np.corrcoef(X_.T)
corr = pd.DataFrame(corr, index=features, columns=features)

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(7, 70, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
feature_imp={
'aspect': 0.38636324856410287,
# 'curvature-plan': 0.022507389222992558,
# 'curvature-prof': 0.032563402846718686,
'curvature': 0.019680834593444985,
'elevation': 0.09614325902426059,
'morphology': 0.1658324173616787,
'slope': 0.04549242135125633,
'tree-cover': 0.23141702703554537}

In [ ]:
np.random.seed(19680801)


plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
y_pos = np.arange(len(feature_imp))
performance = [val for k,val in feature_imp.items()]

ax.barh(y_pos, performance, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(feature_imp.keys())
ax.invert_yaxis()  # labels read top-to-bottom
#ax.set_xlabel('Performance')
#ax.set_title('How fast do you want to go today?')


In [ ]:
corr

In [ ]:
from yellowbrick.features.radviz import radviz

radviz(pd.DataFrame(X_, columns=features), y, classes=[0, 1], alpha=0.3)

In [ ]:
from sklearn import manifold
from sklearn.metrics import euclidean_distances
import matplotlib.pyplot as plt


In [ ]:
inds = np.random.randint(0, len(y), size=1000)
y_=y[inds]

In [ ]:
similarities = euclidean_distances(X_[inds,:-1])

In [ ]:
seed=42
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=-1)
pos = mds.fit(similarities).embedding_

nmds = manifold.MDS(n_components=2, metric=False, max_iter=3000, eps=1e-12,
                    dissimilarity="precomputed", random_state=seed, n_jobs=-1,
                    n_init=1)
npos = nmds.fit_transform(similarities, init=pos)

In [ ]:
s = 100
plt.scatter(npos[y_==0, 0], npos[y_==0, 1], color='darkorange', s=s, lw=0, label='NMDS')
plt.scatter(npos[y_==1, 0], npos[y_==1, 1], color='blue', s=s, lw=0, label='NMDS')
plt.legend(scatterpoints=1, loc='best', shadow=False)

In [ ]:
s = 100
plt.scatter(pos[y_==0, 0], pos[y_==0, 1], color='darkorange', s=s, lw=0, label='MDS')
plt.scatter(pos[y_==1, 0], pos[y_==1, 1], color='blue', s=s, lw=0, label='MDS')
plt.legend(scatterpoints=1, loc='best', shadow=False)

Построим-ка одно деревце решений

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
X_ = StandardScaler().fit_transform(X[:, :-1])
y = y.astype(int)
clf = DecisionTreeClassifier(max_depth=5, random_state=0, class_weight='balanced')
clf.fit(X[:, :-1], y)

In [ ]:
from sklearn.tree import export_graphviz
import pydotplus

# Export resulting tree to DOT source code string
dot_data = export_graphviz(clf,
                           feature_names=features[:-1],
                           out_file=None,
                           filled=True,
                           rounded=True,proportion=True)

#Export to pdf
pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.write_pdf('tree.pdf')

Cross-tab between values 

In [ ]:
from itertools import combinations

#for a, b in combinations(range(len(features)-1), 2):
aspect_bins = np.histogram(X[:,0], bins=10)[1]

cst = pd.crosstab(np.digitize(X[:,0], aspect_bins), X[:, 5])[:-1]

In [ ]:
print(cst)

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(cst)

In [ ]:
#features = ['aspect', 'curvature', 'elevation', 'morphology', 'slope', 'tree-cover', 'patches']

In [ ]:
X_.shape

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df = pd.DataFrame(X_, columns=features)
df.morphology = df.morphology.astype(int)
df['y'] = y
df.y = df.y.map({True: 'W', False: 'F'})
#df = df[df['tree-cover'] > 20]

In [ ]:
df

In [ ]:
box_features = ['aspect',  'curvature', 'elevation', 'slope', 'tree-cover']

In [ ]:
for feature in box_features:
    plt.figure(figsize=(15,10))
    sns.set(font_scale=2)
    sns.boxplot(x="morphology", y=feature,
                hue="y", palette= ["#8de5a1", "#a1c9f4"],
                data=df, fliersize=0)
    plt.legend([],[], frameon=False)
    if feature.startswith('curvature'):
         plt.gca().set(ylim=(-5, 5))
    if feature.startswith('curvature-plan'):
         plt.gca().set(ylim=(-3.5, 3.5))
    if feature.startswith('curvature-prof'):
         plt.gca().set(ylim=(-3.0, 3.0))
    if feature.startswith('slope'):
         plt.gca().set(ylim=(-5, 45))
    if feature.startswith('tree-cover'):
         plt.gca().set(ylim=(40, 105))
    plt.gcf().savefig(f'{feature}.png', dpi=300)
    
    

In [ ]:
df['curvature'][(df.morphology == 7)*(df.y == 'F')]

In [ ]:
from scipy import stats as st

In [ ]:
for feature in box_features:
    for val in np.unique(df.morphology):
        a = df[feature][(df.y=='F')&(df.morphology == val)]
        b = df[feature][(df.y=='W')&(df.morphology == val)]
        print(f"{feature}, morphology = {val}: {st.mannwhitneyu(a, b)}")

In [ ]:
df

In [ ]:
df = pd.DataFrame(X, columns=features)
df.morphology = df.morphology.astype(int)
df['y'] = y
df.y = df.y.map({True: 'W', False: 'F'})
df = df[df['tree-cover'] > 20]
df['area'] = np.digitize(df['tree-cover'].values, [0, 40, 80, 300])


In [ ]:
for feature in box_features:
    plt.figure(figsize=(15,10))
    sns.set(font_scale=2)
    sns.boxplot(x="morphology", y=feature,
                hue="area", palette= ["#8de5a1", "#a1c9f4" , "coral"],
                data=df, fliersize=0)
    plt.legend([],[], frameon=False)
    if feature.startswith('curvature'):
         plt.gca().set(ylim=(-5, 5))
    if feature.startswith('curvature-plan'):
         plt.gca().set(ylim=(-3.5, 3.5))
    if feature.startswith('curvature-prof'):
         plt.gca().set(ylim=(-3.0, 3.0))
    if feature.startswith('slope'):
         plt.gca().set(ylim=(-5, 45))
    if feature.startswith('tree-cover'):
         plt.gca().set(ylim=(40, 105))
    plt.gcf().savefig(f'{feature}-ss3.png', dpi=300)
    

In [ ]:
df